# Using R biomart client to download all known genes (for grch37)

In [ ]:
%load_ext rpy2.ipython

Things the user needs to set:

In [ ]:
# The script will create a new platform annotation file with this name.
NEW_PLATFORM_NAME = 'qDNAseq_hg19'
# What reference assembly is the data on: (hg18, hg19, hg38)
REFERENCE_BUILD = 'hg19'
# Select the platform file you want to map from.
TM_PLATFORM_INPUT_FILE = './' 
# File with the data that needs to be remapped.
TM_SAMPLES_FILE = './180k-Cell-line_samples.txt'
# An indicator for the Call columns
CALL_INDICATOR = '.flag'
# If True, then the mode will be used for genes with many overlapping segments
MODE_FLAG = True

Set the name for the to-be-created platform and send the variables to R.

In [ ]:
PLATFORM_DEFINITION_FILE = NEW_PLATFORM_NAME + '.txt'
%R -i NEW_PLATFORM_NAME
%R -i PLATFORM_DEFINITION_FILE

This changes the host for biomart (the script currently works for hg18 and hg19). 

In [ ]:
host_mapping = {'hg18' : 'may2009.archive.ensembl.org',
                'hg19' : 'grch37.ensembl.org',
                'hg38' : 'www.ensembl.org'}

mart_host = host_mapping[REFERENCE_BUILD]
%R -i mart_host

This checks if biomart is installed and tries to install it. This likely has to be run from inside R terminal though.

In [ ]:
%%R
if (!is.element('biomaRt', installed.packages()[,1])) {
    print('biomaRt not found. Trying to download.')
    source("https://bioconductor.org/biocLite.R")
    biocLite("biomaRt")
}

In [ ]:
%%R

# ### Using ensembl on GRCh37 -----------------------------------------------
library(biomaRt)
mart <- useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                dataset = "hsapiens_gene_ensembl",
                host=mart_host)


# Only use standard human chromosomes
normal.chroms <- c(1:22, "X", "Y", "M")

# # Get the coordinates through biomart and merge with platform -------------
entrez_list <- getBM(attributes = c("chromosome_name", "start_position", "end_position", 
                                    "band", 'hgnc_symbol','entrezgene'),
                        filter = 'chromosome_name',
                        values = normal.chroms,
                        mart = mart)

# Only keep entries with both HGNC symbol and Entrez gene ID
entrez_list <- entrez_list[which(!is.na(entrez_list$entrezgene) & entrez_list$hgnc_symbol != ''),]

# Deduplicate list from hgnc symbols
entrez_list <- entrez_list[order(entrez_list$entrezgene),]
entrez_list <- entrez_list[!duplicated(entrez_list$hgnc_symbol),]

# Sorting based on chromomome and start position
entrez_list <- entrez_list[order(entrez_list$chromosome_name, entrez_list$start_position),]

### Create platform file
biomart_entrez_platform <- data.frame(  "GPL_ID" = NEW_PLATFORM_NAME,
                                        "REGION_NAME" = entrez_list$hgnc_symbol,
                                        "CHR" = entrez_list$chromosome_name,
                                        "START_BP" = as.integer(entrez_list$start_position),
                                        "END_BP" = as.integer(entrez_list$end_position),
                                        "NUM_PROBES" = '',
                                        "CYTOBAND" = entrez_list$band,
                                        "GENE_SYMBOL" = entrez_list$hgnc_symbol,
                                        "GENE_ID" = entrez_list$entrezgene,
                                        "ORGANISM" = 'Homo sapiens'
                                )

write.table(biomart_entrez_platform, file = PLATFORM_DEFINITION_FILE, sep='\t', row.names = FALSE)

# Load custom platform file, and map segements to appropriate values

First load a tranSMART platform file, can be created from any source (eg. biomart R script). Convert the X and Y chromosomes into int for faster comparison

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

platform_raw = pd.read_table(PLATFORM_DEFINITION_FILE)
platform = platform_raw.replace(to_replace='X', value=23)
platform.replace(to_replace='Y', value=24, inplace=True)
platform[['CHR', 'START_BP', 'END_BP']] = platform[['CHR', 'START_BP', 'END_BP']].astype(int)

Load the platform definition you want to map from.

In [ ]:
platform_input = pd.read_table(TM_PLATFORM_INPUT_FILE)
regions = platform_input.ix[:,[2,3,4]]

regions.ix[:,0].replace(to_replace='X', value=23, inplace=True)
regions.ix[:,0].replace(to_replace='Y', value=24, inplace=True)
regions.columns = 'chromosome', 'start', 'end'
regions = regions.astype(int)

This function takes the tranSMART platform file and a segment table as input. The segment table has 4 columns: chr, start, end, value.


In [ ]:
def find_overlapping_segments(chrom, start, end):
    selected_segments_index = regions.loc[((regions.chromosome == chrom) &
                            (regions.end > start) &
                            (regions.start < end))].index
    
    if len(selected_segments_index) > 0:
        return selected_segments_index
    else:
        return None
        
def map_multiple_segments_to_gene(platform):    
    overlap = platform.apply(lambda x: find_overlapping_segments(x.iloc[2], x.iloc[3], x.iloc[4]), 
                            axis=1)
    return overlap

overlap = map_multiple_segments_to_gene(platform)

Now the overlapping genomic regions have been calculated. We can map the data from one to the other. First, though, the file with the data that needs to be mapped has to be loaded.

In [ ]:
segments = pd.read_table(TM_SAMPLES_FILE)
segments_region_column = segments.columns[0]

Columns that contain the flags will be identified.

In [ ]:
col_names_contain_flag = segments.columns.str.contains(CALL_INDICATOR)
column_that_contains_flag = segments.columns[col_names_contain_flag]

In [ ]:
def map_index_to_region_ids(gene):
    mappings = list()
    for id in gene:
        mappings += [platform_input.loc[id]['REGION_NAME']]
    return mappings

def return_mean(segments, mapping):
    mean_values = segments[segments[segments_region_column].isin(mapping)].mean()
    if len(mapping) >= 2 and MODE_FLAG:
        mean_values[column_that_contains_flag] = (segments[segments[segments_region_column] \
                                    .isin(mapping)][column_that_contains_flag]).apply(lambda x: int(stats.mode(x)[0]))
    return mean_values

only_scores = overlap[~overlap.isnull()]
region_id_mapping = only_scores.apply(map_index_to_region_ids)

remapped_regions = region_id_mapping.apply(lambda x: return_mean(segments, x))

new_df = pd.DataFrame(columns=segments.columns,
                      data=remapped_regions)

## Ugly way to add back the lost region_id's
new_df[segments_region_column] = platform.REGION_NAME


When not taking the mode of flag then convert the flag back to int.

In [ ]:
if not MODE_FLAG:
    new_df[column_that_contains_flag] = np.rint(new_df[column_that_contains_flag].astype(float)).astype(int)

Write table to disk with "gene_mapped.txt" added

In [ ]:
remapped_name = TM_SAMPLES_FILE + '_gene_mapped.txt'
new_df.to_csv(remapped_name, sep='\t', float_format='%.3f', index=False)